In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [5]:
REBUILD_DATA = False

In [4]:
class DogsvsCats():
    IMG_SIZE = 50
    CATS = 'PetImages/Cat/'
    DOGS = 'PetImages/Dog/'
    LABELS = {CATS:0, DOGS:1}
    training_data = []
    catcount = 0
    dogcount = 0
    
    def make_train_data(self):
        for label in self.LABELS:
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
                    
                    if label == self.CATS:
                        self.catcount += 1
                    elif label == self.DOGS:
                        self.dogcount += 1
                        
                except Exception as e:
                    pass
                
        np.random.shuffle(self.training_data)
        np.save('training_data.npy', self.training_data)
        print('cats: ', self.catcount)
        print('dogs: ', self.dogcount)
        
if REBUILD_DATA:
    dogsvcats = DogsvsCats()
    dogsvcats.make_train_data()
    

100%|██████████████████████████████████████████████████████████████████████████| 12501/12501 [00:11<00:00, 1048.08it/s]
D:\Anaconda3\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


cats:  12476
dogs:  12470


In [6]:
training_data = np.load('training_data.npy', allow_pickle = True)
print(len(training_data))

24946


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [1]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear,512)
        self.fc2 = nn.Linear(512, 2)
    
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

NameError: name 'nn' is not defined

In [9]:
net = Net()

D:\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [10]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_func = nn.MSELoss()

In [11]:
X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

val_pct = 0.1
val_size = int(len(X)*val_pct)

In [12]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

In [15]:
batch_size = 100
epochs = 1
for epoch in range(epochs):
    for i in tqdm(range(0, len(train_X), batch_size)):
        batch_X = train_X[i: i+batch_size].view(-1, 1, 50, 50)
        batch_y = train_y[i: i+batch_size]
        
        net.zero_grad()
        
        outputs = net(batch_X)
        loss = loss_func(outputs, batch_y)
        loss.backward()
        optimizer.step()
print(loss)

100%|████████████████████████████████████████████████████████████████████████████████| 225/225 [00:42<00:00,  5.26it/s]

tensor(0.2502, grad_fn=<MseLossBackward>)


In [17]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1,1,50,50))[0]
        predicited_class = torch.argmax(net_out)
        if predicited_class == real_class:
            correct += 1
        total += 1
print('Acuracy: ', round((correct/total),3))

100%|█████████████████████████████████████████████████████████████████████████████| 2494/2494 [00:03<00:00, 766.98it/s]

Acuracy:  0.512
